# 라이브러리 import

In [ ]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score, classification_report, f1_score, roc_auc_score,
    precision_recall_curve, auc
)

# 데이터 불러오기

In [ ]:
train_df = pd.read_csv("openworld_train.csv")
test_df = pd.read_csv("openworld_test.csv")

print(train_df.shape)
print(test_df.shape)

(20300, 27)
(8700, 27)


# Feature/Label 분리

In [ ]:
target_col = train_df.columns[-1]

train_df[target_col] = train_df[target_col].apply(lambda x: 0 if x == 95 else 1)
test_df[target_col]  = test_df[target_col].apply(lambda x: 0 if x == 95 else 1)

X_train = train_df.drop(columns=[target_col]).values
y_train = train_df[target_col].astype(np.float32).values
X_test  = test_df.drop(columns=[target_col]).values
y_test  = test_df[target_col].astype(np.float32).values

print(X_train.shape)
print(X_test.shape)

(20300, 26)
(8700, 26)


# 베이스라인 학습

In [ ]:
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)
svm_model.fit(X_train, y_train)

SVC(probability=True, random_state=42)

# 평가

In [ ]:
# test evaluation
test_pred = svm_model.predict(X_test)
test_pred_proba = svm_model.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, test_pred)
test_f1_macro = f1_score(y_test, test_pred, average='macro')
test_f1_micro = f1_score(y_test, test_pred, average='micro')
test_f1_weighted = f1_score(y_test, test_pred, average='weighted')

# ROC-AUC (binary)
test_roc_auc = roc_auc_score(y_test, test_pred_proba)

# PR-AUC
prec, rec, _ = precision_recall_curve(y_test, test_pred_proba)
test_pr_auc = auc(rec, prec)

print(f"Accuracy        : {test_acc:.4f}")
print(f"F1 (macro)      : {test_f1_macro:.4f}")
print(f"F1 (micro)      : {test_f1_micro:.4f}")
print(f"F1 (weighted)   : {test_f1_weighted:.4f}")
print(f"ROC-AUC         : {test_roc_auc:.4f}")
print(f"PR-AUC          : {test_pr_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, test_pred, digits=4))

Accuracy        : 0.8292
F1 (macro)      : 0.7977
F1 (micro)      : 0.8292
F1 (weighted)   : 0.8225
ROC-AUC         : 0.8957
PR-AUC          : 0.9340

Classification Report:
              precision    recall  f1-score   support

         0.0     0.8341    0.6300    0.7178      3000
         1.0     0.8275    0.9340    0.8775      5700

    accuracy                         0.8292      8700
   macro avg     0.8308    0.7820    0.7977      8700
weighted avg     0.8298    0.8292    0.8225      8700

